## Demo: Critical Path

This Notebook shows how the critical path can be examined after an OpenCLSim simulation. The basic steps to are:

* Import libraries
* Setup a basic simulation based on the Notebook '24 - Cutters and barges'
* Demonstration of critical path analysis
* Gantt plot with the critical activities marked

#### 0. Import libraries

The general - better known - libraries.

In [1]:
import datetime, time
import numpy as np
import simpy
import sys
import shapely.geometry
import pandas as pd
from IPython.display import display

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

Inspecting the critical path requires importing ``CpLog`` from the plot subpackage.

In [2]:
from openclsim.plot.critical_path import CpLog

#### 1. Setting up a basic simulation

This section executes a simple simulation with cutters and barges.

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

In [4]:
def run(NR_BARGES, total_amount):
    simulation_start = 0
    my_env = simpy.Environment(initial_time=simulation_start)
    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)

    data_from_site = {"env": my_env,
                      "name": "from_site",
                      "geometry": location_from_site,
                      "capacity": total_amount,
                      "level": total_amount,
                      "nr_resources":1
                     }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {"env": my_env,
                    "name": "to_site",
                    "geometry": location_to_site,
                    "capacity": total_amount,
                    "level": 0,
                    "nr_resources":4
                   }
    to_site = Site(**data_to_site)

    vessels = {}

    for i in range(NR_BARGES):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site, 
            capacity=10,
            compute_v=lambda x: 10
        )
    cutter = TransportProcessingResource(
        env=my_env,
        name=f"cutter",
        geometry=location_from_site, 
        capacity=10,
        compute_v=lambda x: 10
    )
    vessels['cutter'] = cutter
    

    activities = {}
    for i in range(NR_BARGES):
        amount = np.random.randint(4,6) # handle loading
        duration=np.random.randint(2000,3000) # sailing and unloading

        requested_resources={}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[model.SequentialActivity(
                env=my_env,
                name=f"sequential_activity_subcycle{i}",
                registry=registry,
                sub_processes=[
                    model.BasicActivity(
                        env=my_env,
                        name=f"basic activity:"+vessels[f"vessel{i}"].name,
                        registry=registry,
                        duration=duration,
                        additional_logs=[vessels[f"vessel{i}"]],
                    ),
                    model.MoveActivity(
                        env=my_env,
                        name=f"sailing empty:"+vessels[f"vessel{i}"].name,
                        registry=registry,
                        mover=vessels[f"vessel{i}"],
                        destination=from_site,
                        duration=duration,
                    ),
                    model.ShiftAmountActivity(
                        env=my_env,
                        name=f"loading:"+vessels[f"vessel{i}"].name,
                        registry=registry,
                        processor=cutter,
                        origin=from_site,
                        destination=vessels[f"vessel{i}"],
                        amount=amount,
                        duration=500*amount,
                        requested_resources=requested_resources,
                    ),
                    model.MoveActivity(
                        env=my_env,
                        name=f"sailing full:"+vessels[f"vessel{i}"].name,
                        registry=registry,
                        mover=vessels[f"vessel{i}"],
                        destination=to_site,
                        duration=duration,
                    ),
                    model.ShiftAmountActivity(
                        env=my_env,
                        name=f"unloading:"+vessels[f"vessel{i}"].name,
                        registry=registry,
                        processor=vessels[f"vessel{i}"],
                        origin=vessels[f"vessel{i}"],
                        destination=to_site,
                        amount=amount,
                        duration=duration,
                        requested_resources=requested_resources,
                    ),
                ],
            )],
            condition_event=[
                {
                    "type": "container", 
                    "concept": to_site, 
                    "state": "full",
                    "id_":"default_reservations"
                }
            ],
        )

    model.register_processes(list(activities.values()))
    my_env.run()
    
    return {
        "vessels": vessels,
        "activities":activities,
        "from_site":from_site,
        "to_site":to_site,
    }


In [5]:
%%time

res = run(3,100)
vessels = res['vessels']
activities = res['activities']
cutter = vessels['cutter']
to_site = res['to_site']
from_site = res['from_site']

Wall time: 53.4 ms


In [6]:
fig = plot.get_gantt_chart([*vessels.values(), to_site], id_map=[activities[x] for x in activities])

In [7]:
fig = plot.get_gantt_chart([from_site, to_site, cutter], id_map=[activities[x] for x in activities])

#### 2. Critical path

Functionalities for the analysis of the critical path through activities of a simulation are stored in ``openclsim.plot.critical_path``. The main object ``CpLog`` (read as *C*ritical *p*ath *log*) can be used to evaluate the simulated activities from logging and model setup.

The ``CpLog`` is basically a modification of the regular OpenCLSim ``HasLog`` mix-in, for convenience of further analysis.


##### 2a. Initiate a ``CpLog``

The ``CpLog`` is initiated with a list of objects (e.g. vessels and sites), and a list of activities (top level model activities) which are relevant for the critical path:

* In this case we will take as objects the list of vessel objects, and the from and to site.
* The top level activities of the simulation concerns the while activities.

The ``CpLog`` creates an underlying Dataframe with the activities logged. Each activity will be represented in a single row, with a start and end time. An additional ``uuid`` is added which identifies an activity in time.

In [8]:
# define the objects and the activities
my_objects = list(vessels.values()) + [from_site, to_site]
my_activities = list(activities.values())

In [9]:
# create a CpLog object
my_log = CpLog(list_objects=my_objects, list_activities=my_activities)

#### 2b. Examine dependencies between activities

Dependencies between activities are crucial for finding the critical path. For example consider the simple sequential activity of subactivities ``A --> B --> C``, where finishing ``A``, triggers the start of ``B``, and finishing ``B`` triggers the start of ``C``.

This means that we have dependencies between activities. These dependencies are essential when examining the critical path through a sequence of activities. Two separate methods are implemented for assessing these dependencies:

* *log based* Dependencies are 'guessed' based on the log only.
* *model based* Dependencies are investigated based on the model setup.

Please check out the documentation for details on either method. Due to the current logging of OpenCLSim, it is impossible to determine what triggered an activity with 100% certainty. 

In [10]:
dependencies_log = my_log.get_dependencies_log_based()

In [11]:
dependencies_model = my_log.get_dependencies_model_based()

#### 2b. Find the critical path through activities

The critical path can be examined from the ``CpLog`` object with a given list of dependencies derived from it.

This method will return the reshaped log of activities, with their unique idendifyer in time in the column named ``cp_activity_id``. An additional boolean column ``is_critical`` is added indicating whether this particular instance of the activity was found to be on the critical path.

The usage of this method is shown below.

In [12]:
cp_log_based = my_log.mark_critical_activities(dependencies_log)

In [13]:
cp_model_based = my_log.mark_critical_activities(dependencies_model)

#### 2c. Visualize the critical path

Once extracted, the critical path can be visualized in a Gantt chart. The ``plot.get_gantt_chart()`` supports the argument ``critical_path``. This argument takes the output of the method ``CpLog.mark_critical_activities()``. The activities on the critical path are marked with a red line.

The results based on the log-based and model-based methods are shown below.

In [14]:
fig = plot.get_gantt_chart(concepts=my_objects, critical_path=cp_log_based)

In [15]:
fig = plot.get_gantt_chart(concepts=my_objects, critical_path=cp_model_based)